In [1]:
import Tools
import pandas as pd
import index_generate
from PIL import Image
import os
import Onlyclosemodel as OC
import Withidxmodel as WD

#去除因FutureWarning 而無法顯示dataframe的bug，要執行兩次CELL ，我不知道怎麼解決...
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5)) #returns False, without Warning
import matplotlib.pyplot as plt
%matplotlib notebook

class buyer():
    state = 0
    fund = 2000000
    stock = 0
    total = fund + stock
    stockcode = '0'

    df_raw=pd.DataFrame()
    df_index=pd.DataFrame()
    min_maxdf=pd.DataFrame()

    df_index_V2=pd.DataFrame()
    min_maxdf_V2=pd.DataFrame()

    List_ans=np.array(0)
    List_ans_V3=np.array(0)
    List_ans_V4=np.array(0)
    List_price=np.array(0)


    def __init__(self,stockcode):
        self.stockcode=stockcode
        self.df_raw,self.df_index =Tools.checkCodeInDir(self.stockcode) #檢查是否在檔案中，並取得rawdata

        self.min_maxdf=index_generate.Normalize_pd(self.df_index)# 計算Normalize

        self.df_index_V2=index_generate.stock_index_generator_V2(self.df_index,self.stockcode)
        self.min_maxdf_V2=index_generate.Normalize_pd_V2(self.df_index_V2)# 計算Normalize

        self.List_ans, self.List_price,self.List_ans_V3=Tools.get_list_ans(self.stockcode)#計算label答案
        self.List_ans_V4,self.List_price=Tools.get_list_ans_V4(self.stockcode)
    
    def getImg(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=19
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=19
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output\\'+self.stockcode+'\\'+str(self.List_ans[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output/'+self.stockcode+'/'+str(self.List_ans[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
        
    def getImg_V2(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=11
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg_V2(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=11
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output_V2\\'+self.stockcode+'\\'+str(self.List_ans[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output_V2/'+self.stockcode+'/'+str(self.List_ans[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
    def getImg_V3(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=30
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg_V3(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=60
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output_V3\\'+self.stockcode+'\\'+str(self.List_ans_V3[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output_V3/'+self.stockcode+'/'+str(self.List_ans_V3[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
    def saveImg_V4(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=19
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output_V4\\'+self.stockcode+'\\'+str(self.List_ans_V4[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output_V4/'+self.stockcode+'/'+str(self.List_ans_V4[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
    
    
    def get_ans_price(self):
        return self.List_ans, self.List_price

    def buystock(self,price):
        self.stock=(self.fund/1.003)/price
        self.fund=0
        self.total=self.fund+(self.stock*price)
        self.state=1
        print("buy")
    def selstock(self,price):   
        self.fund=self.stock*price*0.997
        self.stock=0
        self.total=self.fund+(self.stock*price)
        self.state=0
        print("sell")
    def holstock(self,price):
        self.total=self.fund+(self.stock*price)
        print("hold")
    def make_choice(self,inp,price):
        if (self.state==0 and inp==1):
            self.buystock(price)
        elif(self.state==1 and inp==-1):
            self.selstock(price)
        else :
            self.holstock(price)
    def train(self,model='ALL',train=True):
        if model == 'ALL' :
            OC.stock_train(self.stockcode,self.df_index,train)
            WD.stock_train(self.stockcode,self.df_index,train)
        elif model == 'OC':
            OC.stock_train(self.stockcode,self.df_index,train)
        elif model == 'WD':
            WD.stock_train(self.stockcode,self.df_index,train)
        else :
            print('Error: no fucking input')

Using TensorFlow backend.
False


In [2]:
stockcodes=["2330","2317","2454","3008","1301","2412","1303","2891","1216","2882","2881","2886","2308","2884","1326","2002","3711","2892","2885","2207","1101","2880","5880","3045","2303","2474","2912","2382","5876","5871","2357","2327","2887","2801","2890","4938","2883","6505","2888","1402","4904","2395","1102","2301","9904","2633","2105","2823","9910","2408"]

for i in stockcodes:
    Bob=buyer(i)

found data [2330.json]at : ./stock_data
found data [2330_index.json]at : ./stock_data_index
found data [2317.json]at : ./stock_data
found data [2317_index.json]at : ./stock_data_index
[2454.json] is not be found ,crawling data from ''https://www.twse.com.tw/exchangeReport/STOCK_DAY?'',
please wait
20100101
20100201
20100301
20100401
20100501
20100601
20100701


In [2]:
Tom=buyer('0050')
Tom.df_index

found data [0050.json]at : ./stock_data
found data [0050_index.json]at : ./stock_data_index


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,RSI_12,Ans,K,D,RSI_DIF,BIAS,WMR,EMA_12,EMA_26,MACD,psy_6,MTM_6,SAR_6,DM+(DMI),DM-(DMI),TR(DMI),+DI(DMI),-DI(DMI),ADX(DMI),CDP,AH,NH,NL,AL,Trend
0,99/01/22,29477562,1612081382,55.20,55.20,54.35,54.40,-1.55,3929,8.860759,24.031008,-1,31.640453,44.700091,0,-3.104326,85.542169,56.521162,56.688473,0.113084,33.333333,-3.05,57.862422,-1.20,-1.40,1.60,-2.336449,-2.803738,-9.090909,56.0125,56.6625,56.275,55.625,55.3625,1
1,99/01/25,12845252,695785374,53.95,54.45,53.80,54.20,-0.20,2147,0.000000,20.529801,1,32.058506,40.486651,0,-2.254283,70.707071,56.164060,56.504142,0.052662,16.666667,-3.60,57.721925,-0.75,-0.55,0.65,-3.011604,-3.590695,-9.068063,54.5875,55.4375,54.825,53.975,53.7375,2
2,99/01/26,26395774,1416284559,54.15,54.55,52.75,52.95,-1.25,4558,0.000000,16.551724,-1,21.849549,34.274905,0,-3.363316,95.000000,55.669589,56.240872,-0.030531,0.000000,-4.55,57.565048,0.10,-1.05,1.80,-4.230915,-4.710095,-8.803155,54.1625,54.8125,54.525,53.875,53.5125,3
3,99/01/27,14855839,787198926,52.95,53.30,52.70,52.90,-0.05,2182,23.000000,26.666667,1,14.567094,27.706291,0,-2.770119,95.294118,55.243499,55.993400,-0.126447,0.000000,-3.85,57.372446,-1.25,-0.05,0.60,-5.721424,-6.474500,-8.615417,53.3000,55.1000,53.850,52.050,51.5000,3
4,99/01/28,12622891,676089714,53.10,54.20,53.10,54.05,1.15,1822,20.175439,24.444444,-1,35.264882,30.225570,0,0.238442,55.952381,55.059884,55.849444,-0.214862,0.000000,-2.35,57.185548,0.90,0.40,1.30,-8.084180,-8.640607,-8.237670,52.9500,53.5500,53.200,52.600,52.3500,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,108/01/24,3500506,258832935,73.95,74.15,73.75,74.05,0.35,1363,86.538462,81.147541,1,30.054097,50.017913,0,-1.229040,86.885246,74.787302,74.991420,-0.279373,33.333333,-1.50,77.284078,0.15,0.10,0.45,-3.012712,-2.233089,2.350334,73.7625,74.1125,73.875,73.525,73.4125,5
2226,108/01/25,7320778,548115919,74.60,75.10,74.55,75.00,0.95,3322,87.272727,76.530612,1,35.511709,45.182995,0,0.114416,55.737705,74.820025,74.992056,-0.265061,50.000000,-0.55,77.260715,0.95,0.80,1.05,-2.058602,-1.115729,4.304100,74.0000,74.4000,74.250,73.850,73.6000,6
2227,108/01/28,3825143,287814225,75.15,75.45,75.05,75.15,0.15,1795,66.666667,70.093458,-1,57.005657,49.123488,-1,0.592791,50.819672,74.870790,75.003755,-0.247448,66.666667,-0.80,77.238287,0.35,0.50,0.45,-1.299314,-0.278016,8.621558,74.9125,75.4625,75.275,74.725,74.3625,4
2228,108/01/29,4419790,328663492,74.50,74.55,74.20,74.50,-0.65,2251,55.769231,63.541667,1,56.394637,51.546962,0,0.134409,52.777778,74.813745,74.966440,-0.234814,66.666667,-2.00,77.216755,-0.90,-0.85,0.95,-0.439091,0.627320,-32.462157,75.2000,75.6000,75.350,74.950,74.8000,1


In [10]:
# test input get img
# 毛哥要用的時候
# cnn_ans(img)換成你寫好的預測結果

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
import torchvision.models as models

CUDA_DEVICES=0
PATH_TO_WEIGHTS = './Model_all_best.pth'

def cnn_ans(img):
    
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    model=models.resnet101()
    model = torch.load(PATH_TO_WEIGHTS)
    model = model.cuda(CUDA_DEVICES)
    model.eval()
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():

        # data_loader的格式 (batch_size, channels, height, width)
        # 因為這裡沒用 data_loader() 當媒介餵資料，所以缺少batch維度，要用unsqueeze在最前面增加一個維度。
        
        inputs = data_transform(img).unsqueeze(0)
        inputs = Variable(inputs.cuda())  # CUDA_DEVICES))

        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        tmp = outputs.data.cpu().numpy()
        _, predictions = torch.max(outputs.data, 1)
        
        change_ans = [-1,0,1]
        
        predictions = change_ans[predictions.int()]
               
        print("Predictions: ", predictions)
    
    
    return predictions


John=buyer('0050')
for i in range(19,John.List_ans.size-1):
    img=John.getImg(i)
    ans=cnn_ans(img)
    John.make_choice(ans,float(John.List_price[i-10]))
print(John.total)


 0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predi

In [2]:
# test input savelabel

Bob=buyer('0053')
Bob.min_maxdf

for i in range(19,Bob.List_ans.size-1):
    img=Bob.saveImg(i)
    Bob.make_choice(Bob.List_ans[i],float(Bob.List_price[i]))



ing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Financ

In [8]:
Wei=buyer('0050')
for i in range(60,Wei.List_ans.size-1):
    img=Wei.saveImg_V4(i)
    Wei.make_choice(Wei.List_ans_V4[i],float(Wei.List_price[i-10]))

ce/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V4/0050/0/
hold
C:/Users

In [8]:
Bob=buyer('0054')

for i in range(19,Bob.List_ans.size-1):
    img=Bob.saveImg_V2(i)
    Bob.make_choice(Bob.List_ans[i],float(Bob.List_price[i]))


image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zi